In [ ]:
ad = dbutils.secrets.get('Healthcare2.0-SecretScope', 'healthcare-ad')
sp = dbutils.secrets.get('Healthcare2.0-SecretScope', 'Healthcare-sp')
se = dbutils.secrets.get('Healthcare2.0-SecretScope', 'healthcare-secret')


spark.conf.set("fs.azure.account.auth.type.#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net", sp)
spark.conf.set("fs.azure.account.oauth2.client.secret.#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net", se)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net", "https://login.microsoftonline.com/" + ad + "/oauth2/token")

In [ ]:
import dlt
from pyspark.sql.functions import struct
from pyspark.sql.functions import col
from pyspark.sql.types import DateType
from pyspark.sql.types import *
import pyspark.sql.functions as f
from pyspark.sql.functions import udf
from pyspark.sql.functions import *
import re
import itertools
import numpy as np
from delta.tables import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DateType, TimestampType, DecimalType
from pyspark.sql.functions import regexp_replace

In [ ]:
# test_input_file = "abfss://data-source@#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net/Bed Occupancy and Availability Report/OccupancyHealthcareData.csv"
# df=spark.read.format("csv").option("header",True).load(test_input_file)
# df.show()
# # readingDateTimeUTC	tempC	humidity	pressure	pM25	pM10	pM1	aqi	merge	hour	Date	Time	Column	LocationKey	Time - Copy	Month1	Year	week	aqiLabel	AQILabelID	latitude1	longitude1


In [ ]:
GlbBedOccupancy_file = "abfss://data-source@#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net/GlobalOverviewTiles/GlobalOverviewReport_BedOccupancy.csv"
from pyspark.sql.functions import col

@dlt.table()
def GlbBedOccupancy_bronze():
  return (spark.read.format("csv").option("header",True).load(GlbBedOccupancy_file))

@dlt.table()
def GlbBedOccupancy_silver():
  return (dlt.read("GlbBedOccupancy_bronze").select(col("city").alias("City"),col("MonthNumber").alias("MonthNumber"),col("BedOccupancyRate").alias("BedOccupancyRate")))

@dlt.table()
def GlbBedOccupancy_gold():
  return (dlt.read("GlbBedOccupancy_silver"))

In [ ]:
GlbMarginRate_file = "abfss://data-source@#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net/GlobalOverviewTiles/GlobalOverviewReport_MarginRate.csv"
from pyspark.sql.functions import col

@dlt.table()
def GlbMarginRate_bronze():
  return (spark.read.format("csv").option("header",True).load(GlbMarginRate_file))

@dlt.table()
def GlbMarginRate_silver():
  return (dlt.read("GlbMarginRate_bronze").select(col("city").alias("City"),col("marginPercent").alias("MarginPercent"),col("month").alias("MonthName")))

@dlt.table()
def GlbMarginRate_gold():
  return (dlt.read("GlbMarginRate_silver"))

In [ ]:
GlbPatientExperience_file = "abfss://data-source@#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net/GlobalOverviewTiles/GlobalOverviewReport_PatientExperience.csv"
from pyspark.sql.functions import col

@dlt.table()
def GlbPatientExperience_bronze():
  return (spark.read.format("csv").option("header",True).load(GlbPatientExperience_file))

@dlt.table()
def GlbPatientExperience_silver():
  return (dlt.read("GlbPatientExperience_bronze").select(col("city").alias("City"),col("PatientExperience").alias("PatientExperience"),col("month").alias("MonthName")))

@dlt.table()
def GlbPatientExperience_gold():
  return (dlt.read("GlbPatientExperience_silver"))

In [ ]:
%sql

select * from csv.`abfss://data-source@#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net/Bed Occupancy and Availability Report/OccupancyHealthcareData.csv`

In [ ]:
# Supplier_input_file = "abfss://data-source@#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net/Bed Occupancy and Availability Report/Healthdata.csv"
# df=spark.read.format("csv").option("header",True).load(Supplier_input_file)
# df.show()